In [ ]:
# Imports
import openai
import requests
from bs4 import BeautifulSoup
import json
import datetime
import my_target_job
import my_secrets

In [ ]:
# Set OpenAI API key
openai.api_key = my_secrets.OPENAI_API_KEY

In [ ]:
# Get job description from URL

# Send a GET request to the URL
url_response = requests.get(my_target_job.job_description_url)

# Extract the HTML content from the response
html_content = url_response.text

# Create a BeautifulSoup object to parse the HTML
soup = BeautifulSoup(html_content, "html.parser")

# Extract the text from the webpage
job_description = soup.get_text()

# Remove whitespace characters
job_description = " ".join(job_description.split())  

print(job_description)

In [ ]:
# Helper function: Get ChatGPT response from text prompt using API
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = temperature,  # degree of randomness of GPT response
    )
    return response.choices[0].message["content"]